# Rubin LSST DESC DC2: Accessing variable truth information (for stars and SNe) with GCRCatalogs

**Authors**: Yao-Yuan Mao (@yymao), Joanne Bogart (@JoanneBogart)

This notebook will illustrate the basics of accessing the variable truth information, such as the light curves for stars and SNe.

**Prerequisite (optional)**: The Object Table tutorial provides an introduction to `GCRCatalogs`. It is not strictly needed in this tutorial, but if you have questions about the use of `GCRCatalogs`, please check out the Object Table tutorial.

**Learning objectives**: After going through this notebook, you should be able to:
  1. Know the difference of the various truth catalogs related variable truth information.
  2. Load the light curves for any SNe or stars that are of interset. 

## Before you start

Make sure you have followed the instructions on the [DESC Data Portal](https://lsstdesc-portal.nersc.gov/) to 
download the data files, install `GCRCatalogs`, and set up `root_dir` for `GCRCatalogs`.

In this example notebook, the following files will be needed:
- `lsstdesc-public/dc2/run2.2i-dr6-v4/truth_sn/truth_sn_summary_v1-0-0.parquet`
- `lsstdesc-public/dc2/run2.2i-dr6-v4/truth_sn/truth_sn_variability_v1-0-0.parquet`
- `lsstdesc-public/dc2/run2.2i-dr6-v4/truth_star/truth_star_summary_v1-0-0.parquet`
- `lsstdesc-public/dc2/run2.2i-dr6-v4/truth_star/truth_star_variability_v1-0-0.parquet`

## Import necessary packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord

In [ ]:
import GCRCatalogs
from GCRCatalogs import GCRQuery
from GCRCatalogs.utils import first

In [ ]:
def flux2mag(flux):
    """
    A convenience function to convert flux to magnitude
    """
    with np.errstate(divide="ignore"):
        return 22.5 - 2.5 * np.log10(flux)

## Supernova light curves

We will start with finding light curves for supernovae.  When accessing DC2 variable truth information, one important concept is that the light curves are stored separately from a "summary table". 

In the summary table, each row corresponds to one object (here a supernova), and it contains summary truth information such as coordinates, redshift, and time-averaged fluxes - that is, attributes associated with the object overall, not with a particular observation of it. Hence it does not include light curve values.

We can use the summary table to figure out the IDs of supernovae that we are interested in (e.g., in a certain sky area, in certain redshift ranges), these IDs will later be used to extract light curves.

### Loading supernova summary table

We first load the summary table into `sn_summary_cat` and check out what columns are available.

In [ ]:
sn_summary_cat = GCRCatalogs.load_catalog("desc_dc2_run2.2i_truth_sn_summary")

In [ ]:
sorted(sn_summary_cat.list_all_quantities())

We will now actually load the data into memory. For this tutorial, we will only need the ID (`id`), coordinates (`ra`, `dec`), and redshift (`redshift`). 

Recall that `get_quantities` returns a Python dictionary. We can use `pd.DataFrame` to turn it into a Pandas DataFrame.

In [ ]:
d_summary = pd.DataFrame(sn_summary_cat.get_quantities(["id", "ra", "dec", "redshift"]))
d_summary.head()

Now that we have the summary table, we can use it to select, say supernovae near a certain coordinate or in a certain redshift range. In this tutorial, we will find all supernovae that are within 2 arcmin of (RA, Dec) = (60.5, -36.6) and have a redshift less than 0.8. We will store these supernovae in `objects_to_plot` which we will use later to extract and plot the their light curves.

In [ ]:
selected = SkyCoord(d_summary["ra"].values, d_summary["dec"].values, unit="deg").separation(SkyCoord(60.5, -36.6, unit="deg")).arcmin < 2
selected &= d_summary["redshift"] < 0.8

objects_to_plot = d_summary.iloc[np.flatnonzero(selected)]
objects_to_plot

### Loading supernova light curve (variability) table

Now that we have a list of supernovae that we want to inspect further, we can load the "variability table", which contains the light curves. 

We will load the "variability table" into `sn_light_curve_cat` and check what columns are available. 

In [ ]:
sn_light_curve_cat = GCRCatalogs.load_catalog("desc_dc2_run2.2i_truth_sn_variability")
sn_light_curve_cat.list_all_quantities()

Unlike the summary table, in the variability table each row corresponds to one "measurement": the flux of a given object, at a given time, with a given filter. Hence, a single supernova is usually represented by many rows.

Here, let's peek the beginning of the variability table. Note that the variability table is a much larger table, and you probably don't want to load the whole thing in the memory. Here we set `return_iterator=True` so that we can look at just the first chunk of the table. 

In [ ]:
d_lc = pd.DataFrame(first(sn_light_curve_cat.get_quantities(sn_light_curve_cat.list_all_quantities(), return_iterator=True)))
d_lc.head()

You can see from the above the the first five rows are all different measurements (at different times and not always with the same filter) for a single object (`MS_9177_2005`). 

Now that we have a basic idea of how the variability table is structured, we can start to search for the light curves for the supernovae that we identified earlier. In short, we need to extract all rows that have IDs overlapping with the IDs in `objects_to_plot`. 

Because the variability table is a large table, we don't want to load the whole thing and then do the search. By specifying `filters` in `get_quantities`, the backend code will do the search on each smaller chunk, which will save the runtime memory. 

The filter (i.e., search query) we will use here looks a bit complicated, but it really just means find all rows whose `id` is in `objects_to_plot["id"].values`: 
```python
GCRQuery((lambda x: np.isin(x, objects_to_plot["id"].values), "id"))
```
Even with this filter, the cell below will take a while to run. If runtime is a concern, you may want to use the SQL database that we also provide to access this information instead.

In [ ]:
d_lc = pd.DataFrame(sn_light_curve_cat.get_quantities(
    ['id', 'MJD', 'bandpass', 'delta_flux'],
    filters=[GCRQuery((lambda x: np.isin(x, objects_to_plot["id"].values), "id"))],
))
d_lc.head()

By now, this `d_lc` DataFrame has all the data for everything in `objects_to_plot`. We would want to split the table by both the object IDs and by bandpass (filter). Pandas' `groupby` function comes in handy!

In [ ]:
d_lc_grouped = d_lc.groupby(["id", "bandpass"]).groups

In [ ]:
fig, ax = plt.subplots(ncols=len(objects_to_plot), figsize=(len(objects_to_plot)*4, 4))

for obj, ax_this in zip(objects_to_plot.itertuples(), ax):
    for i, band in enumerate("ugrizy"):
        indices = d_lc_grouped.get((obj.id, band))
        if indices is not None:
            ax_this.plot(d_lc["MJD"][indices], flux2mag(d_lc["delta_flux"][indices]), 'o:', color=f"C{i}", label=f"${band}$")
    ax_this.legend(ncol=3)
    ax_this.set_title(obj.id)
    ax_this.set_xlabel("MJD")
    ax_this.set_ylabel("magnitude")
    ax_this.set_ylim(25, 10)

fig.tight_layout()        

Now we have the light curves! You might notice that we used `delta_flux` to get the magnitude. Here `delta_flux` is the flux difference to the inifinite time averaged flux value. For SNe, the infinite time-averaged fluxes are always zero, and hence `delta_flux` would just store the actual flux values. As we will see later, for stars we will have to add the infinite time-averaged fluxes to `delta_flux` to get actual flux values.

## Variable star light curves

Now we are going to repeat everything we have done so far, but with variable stars. Just like what we did earlier, we will start with getting the summary information, and use that to identify some variable stars of interest. 

### Loading star summary table

In [ ]:
star_summary_cat = GCRCatalogs.load_catalog("desc_dc2_run2.2i_truth_star_summary")

In [ ]:
sorted(star_summary_cat.list_all_quantities())

Here the fluxes (`flux_*`) in the stars' summary table are time-averaged fluxes over *infinite* time, which we will need when we plot the light curves.

Also, not all stars in this summary table are "variable"; some are rather static. The column `above_threshold` tells us if the star is "variable enough" to be included in the variability table. So here we will also grab it when we are looking for interesting variable stars. 

In [ ]:
d_summary = pd.DataFrame(star_summary_cat.get_quantities(["id", "ra", "dec", "above_threshold"] + [f"flux_{band}" for band in "ugrizy"]))

Another important difference between stars and supernovae is that for stars, their infinite time-averaged fluxes are generally non-zero! We can verify that:

In [ ]:
d_summary[[f"flux_{band}" for band in "ugrizy"]].min()

Like earlier, we will now use the summary table to select variable stars near a certain coordinate. Here, we further select bright stars and stars with enough variability (to ensure that they appear in the variability table). We will store the selected stars in `objects_to_plot` which we will use later to extract and plot the their light curves.

In [ ]:
selected = SkyCoord(d_summary["ra"].values, d_summary["dec"].values, unit="deg").separation(SkyCoord(60.5, -36.6, unit="deg")).arcmin < 2
selected &= d_summary["flux_r"] > 1.0e3
selected &= d_summary["above_threshold"] > 0

objects_to_plot = d_summary.iloc[np.flatnonzero(selected)]
objects_to_plot

### Loading supernova light curve (variability) table

We will now load the "variability table" for stars.

In [ ]:
star_lc_cat = GCRCatalogs.load_catalog("desc_dc2_run2.2i_truth_star_variability")

Again, recall that in the "variability table", each row corresponds to one measurement, and we need to collect all the rows that have `id` values overlapping with `objects_to_plot["id"].values`. 

Also remember that the variability table is a large table (even larger in the star case). We will use the same `filters` trick again to reduce the memory footprint, but the cell below will still result in ~4 GB at peak memory usage. It will take a while to run. If runtime is a concern, you may want to use the SQL database that we also provide to access this information instead.

In [ ]:
d_lc = pd.DataFrame(star_lc_cat.get_quantities(
    ['id', 'MJD', 'bandpass', 'delta_flux'],
    filters=[GCRQuery((lambda x: np.isin(x, objects_to_plot["id"].values), "id"))],
))
d_lc.head()

Once we collect the light curve infomation in `d_lc`. We can use the same `groupby` trick to split the data frame by ID and bandpass, and proceed to make the light curve plot.

In [ ]:
d_lc_grouped = d_lc.groupby(["id", "bandpass"]).groups

In [ ]:
fig, ax = plt.subplots(ncols=len(objects_to_plot), figsize=(len(objects_to_plot)*4, 4))

for obj, ax_this in zip(objects_to_plot.itertuples(), ax.flat):
    for i, band in enumerate("ugrizy"):
        indices = d_lc_grouped.get((obj.id, band))
        if indices is not None:
            ax_this.plot(d_lc["MJD"][indices], flux2mag(getattr(obj, f"flux_{band}") + d_lc["delta_flux"][indices]), 'o:', color=f"C{i}", label=f"${band}$")
    ax_this.legend(ncol=3)
    ax_this.set_title(obj.id)
    ax_this.set_xlabel("MJD")
    ax_this.set_ylabel("magnitude")

fig.tight_layout()        

It's important to note that here we must add the time-averaged `flux_X` to `delta_flux` to obtain the actual flux values. It is expected that the variability of the stars is much smaller than the SNe. 

Because the magnitudes in different filters span a much wider range when compared to the stellar variability, it's somewhat difficult to see the variability from the plot above. To see the variability more clearly, we can subtract out the time-average magnitude (i.e., plotting the difference in magnitude):

In [ ]:
fig, ax = plt.subplots(ncols=len(objects_to_plot), figsize=(len(objects_to_plot)*4, 4))

for obj, ax_this in zip(objects_to_plot.itertuples(), ax.flat):
    for i, band in [(2,"r")]:
        indices = d_lc_grouped.get((obj.id, band))
        if indices is not None:
            ax_this.plot(d_lc["MJD"][indices], flux2mag(getattr(obj, f"flux_{band}") + d_lc["delta_flux"][indices]) - flux2mag(getattr(obj, f"flux_{band}")), 'o:', color=f"C{i}", label=f"${band}$")
    ax_this.legend(ncol=3)
    ax_this.set_title(obj.id)
    ax_this.set_xlabel("MJD")
    ax_this.set_ylabel("$\Delta$ magnitude")
fig.tight_layout()        